In [1]:
import random
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeClassifierCV
import sktime
from sktime.datatypes._panel._convert import from_3d_numpy_to_nested
from sktime.transformations.panel.rocket import Rocket
from sklearn.metrics import classification_report,accuracy_score,precision_score,recall_score


In [2]:
df = pd.read_csv('reconstructed_data.csv') # load the data of EMG signals

df.set_index('sequence',inplace=True) # set the index to the sequence column

X = df.drop('label',axis=1) # drop the label column from the data
Y = df[['label']] # get the label column to be used as the target
#Y = df.drop(df.columns[df.apply(lambda x: x.name != 'label')],axis=1)

In [3]:
X

,0,1,2,3,4,5,6,7,8,9,...,14990,14991,14992,14993,14994,14995,14996,14997,14998,14999
sequence,,,,,,,,,,,,,,,,,,,,,
0,0.100343,0.100350,0.100370,0.100400,0.100435,0.100473,0.100511,0.100550,0.100588,0.100627,...,0.099908,0.099945,0.099982,0.100019,0.100056,0.100093,0.100129,0.100161,0.100185,0.100198
0,0.110339,0.110336,0.110327,0.110314,0.110298,0.110281,0.110264,0.110247,0.110230,0.110213,...,0.110540,0.110523,0.110505,0.110488,0.110471,0.110454,0.110437,0.110423,0.110412,0.110405
0,0.036453,0.036453,0.036452,0.036451,0.036449,0.036447,0.036446,0.036444,0.036442,0.036440,...,0.036475,0.036473,0.036471,0.036469,0.036467,0.036466,0.036464,0.036462,0.036461,0.036460
0,0.020544,0.020545,0.020549,0.020556,0.020563,0.020571,0.020580,0.020588,0.020596,0.020604,...,0.020446,0.020455,0.020463,0.020472,0.020480,0.020488,0.020496,0.020503,0.020509,0.020512
1,0.103840,0.103843,0.103849,0.103859,0.103871,0.103883,0.103896,0.103909,0.103922,0.103935,...,0.103698,0.103710,0.103722,0.103734,0.103746,0.103758,0.103770,0.103780,0.103788,0.103792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1998,0.014313,0.014314,0.014318,0.014323,0.014329,0.014336,0.014343,0.014350,0.014357,0.014364,...,0.014234,0.014241,0.014248,0.014254,0.014261,0.014268,0.014274,0.014280,0.014285,0.014287
1999,0.093110,0.093094,0.093051,0.092987,0.092911,0.092829,0.092746,0.092663,0.092580,0.092497,...,0.094083,0.093999,0.093915,0.093831,0.093747,0.093663,0.093584,0.093513,0.093459,0.093430
1999,0.118611,0.118603,0.118581,0.118549,0.118510,0.118469,0.118427,0.118385,0.118343,0.118301,...,0.119101,0.119059,0.119016,0.118974,0.118932,0.118890,0.118850,0.118814,0.118787,0.118772


In [4]:
Y

,label
sequence,
0,0
0,0
0,0
0,0
1,0
...,...
1998,9
1999,9
1999,9


In [5]:
Xarray = X.to_numpy() # convert the data to numpy array
Xarray = Xarray.reshape(2000,4,-1) # reshape the data to 3D numpy array
Xarray

array([[[0.10034339, 0.10035037, 0.10037013, ..., 0.10016064,
         0.10018479, 0.10019794],
        [0.11033939, 0.11033625, 0.11032735, ..., 0.11042262,
         0.11041153, 0.1104055 ],
        [0.03645344, 0.03645311, 0.03645218, ..., 0.03646228,
         0.03646109, 0.03646044],
        [0.02054354, 0.02054505, 0.02054933, ..., 0.02050337,
         0.02050873, 0.02051165]],

       [[0.10384047, 0.10384279, 0.10384935, ..., 0.10378019,
         0.10378812, 0.10379244],
        [0.10463462, 0.10463483, 0.10463543, ..., 0.10462849,
         0.10462936, 0.10462983],
        [0.03283462, 0.03283406, 0.03283249, ..., 0.03285   ,
         0.03284789, 0.03284675],
        [0.01517424, 0.01517493, 0.0151769 , ..., 0.01515572,
         0.0151582 , 0.01515955]],

       [[0.1147981 , 0.11480091, 0.11480885, ..., 0.11472528,
         0.11473484, 0.11474005],
        [0.10165947, 0.10165994, 0.10166129, ..., 0.10164754,
         0.10164906, 0.1016499 ],
        [0.0358137 , 0.03581332, 0.0

In [6]:
Yarray = Y.to_numpy() # convert the target to numpy array
Yarray = Yarray.reshape(2000,4,-1) # reshape the target to 3D numpy array
Yarray = np.mean(Yarray,axis=1).astype(int) # get the mean of the target and convert it to integer to make it a 2D numpy array

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(Xarray, Yarray, test_size = 0.3, random_state=42) # split the data into training and testing set

In [8]:
Y_train.shape, Y_test.shape # check the shape of the target

((1400, 1), (600, 1))

In [9]:
rocket = Rocket(num_kernels=1000,n_jobs=-1) # create a rocket object. set the number of kernels to 1000(default is 10000) and use all available cores

In [10]:
X_train = from_3d_numpy_to_nested(X_train) # convert the training data to nested format
rocket.fit(X_train) # fit the rocket object to the training data
X_train_transform = rocket.transform(X_train) # transform the training data

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [12]:
classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10)) # create a RidgeClassifier object
classifier.fit(X_train_transform, Y_train) # fit the model into classifier

/opt/miniconda3/envs/bigdata/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:1190: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


RidgeClassifierCV(alphas=array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]))

In [11]:
X_test = from_3d_numpy_to_nested(X_test) # convert the testing data to nested format
X_test_transform = rocket.transform(X_test) # transform the testing data

In [13]:
Y_pred = classifier.predict(X_test_transform) # predict the target of the testing data
print(classification_report(Y_pred,Y_test)) # print the classification report to check the performance of the model

              precision    recall  f1-score   support

           0       0.91      0.88      0.89        56
           1       0.90      0.81      0.86        70
           2       0.92      0.98      0.95        57
           3       0.68      0.80      0.73        50
           4       0.93      0.83      0.88        77
           5       0.93      0.94      0.93        53
           6       0.79      0.85      0.82        71
           7       0.82      0.81      0.82        63
           8       0.66      0.85      0.74        46
           9       0.84      0.63      0.72        57

    accuracy                           0.84       600
   macro avg       0.84      0.84      0.83       600
weighted avg       0.84      0.84      0.84       600

